# AdaBoost
En este notebook presentamos un ejemplo de aplicación del algoritmo AdaBoost.
Se utilizarán los datos de cancer de pecho de sklearn. El objetivo es clasificar el cancer en base a atributos de los tumores


In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn import tree

# Utilizando el dataset breast cancer
cancer = load_breast_cancer()

# dataset en formato tabular
pd.DataFrame(data=cancer.data,
            columns=cancer.feature_names).head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Objetivo: clasificar casos de cáncer de pecho según varios atributos de los tumores.

In [2]:
cancer.data.shape

(569, 30)

In [3]:
# separmos los datos en sets de entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(cancer.data, 
                        cancer.target, random_state=1)

Realizamos un primer clasificador con un arbol de decisión de 2 niveles de profundidad.



In [4]:
# 
# Armando un simple arbol de decisión
treeCLF = DecisionTreeClassifier(max_depth=2, random_state=0)
treeCLF.fit(X_train, y_train)
print('Precisión modelo inicial train/test  {0:.3f}/{1:.3f}'
      .format(treeCLF.score(X_train, y_train), treeCLF.score(X_test, y_test)))

Precisión modelo inicial train/test  0.962/0.888


Este clasificador tuvo un rendimiento bastante bueno, logrando una precisión del 96% con los datos de entrenamiento y del 89% con los datos de evaluación.

**Segunda aproximación: Utilizar AdaBoost para mejorar la precisión**

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Utilizando AdaBoost para aumentar la precisión
ada = AdaBoostClassifier(base_estimator=treeCLF, n_estimators=500, 
                         learning_rate=0.1, random_state=1)
# Ajustando los datos
ada = ada.fit(X_train, y_train)

In [6]:
# prediccion con los datos de entrenamiento, sólo para ver overfitting
y_train_pred = ada.predict(X_train)
ada_train = accuracy_score(y_train, y_train_pred)

In [7]:
y_test_pred = ada.predict(X_test)
ada_test = accuracy_score(y_test, y_test_pred)

In [8]:
print('Precisión modelo con AdaBoost train/test  {0:.3f}/{1:.3f}'
      .format(ada_train, ada_test))

Precisión modelo con AdaBoost train/test  1.000/0.958


In [9]:
# matriz de confusion

# Utilizamos el metodo crosstab para crear la matriz de confusion
confusion_matrix = pd.crosstab(y_test, y_test_pred, rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted   0   1
Actual           
0          51   4
1           2  86


In [10]:
confusion_matrix = pd.crosstab(y_train, y_train_pred, rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted    0    1
Actual             
0          157    0
1            0  269


Vemos que el modelo con AdaBoost logra una precisión del 100% en los datos de entrenamiento y del 96% en los datos de evaluación.

Debemos tener en cuenta que una precisión del 100% sobre los datos de entrenamiento, puede ser un indicio de que el modelo tal vez este sobreajustado. El sobreajuste es uno de los riesgo que suele traer aparejado la utilización de las técnicas de Boosting.

A partir del éxito inicial de AdaBoost, las técnicas de Boosting fueron evolucionando hacia un modelo estadístico más generalizado, tratando el problema como un caso de optimización numérica en dónde el objetivo es minimizar la función de perdida del modelo mediante la adición de los algoritmos de aprendizaje débiles utilizando un procedimiento de optimización del tipo de gradiente descendiente. 

Esta generalización permitió utilizar funciones arbitrarias de pérdida diferenciables, ampliando la técnica más allá de los problemas de clasificación binaria hacia problemas de regresión y de clasificación multi-variable. Esta nueva familia de algoritmos de Boosting se conocen bajo el nombre de Gradient boosting.

## XGBoost en Python
https://xgboost.readthedocs.io/en/stable/

In [15]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression as LR
import numpy as np


clf_xgb = xgb.XGBClassifier(n_estimators=500, max_depth=2, learning_rate=0.1, objective='binary:logistic')
clf_xgb.fit(X_train, y_train)
preds_xgb = clf_xgb.predict(X_test)

C:\Users\amasdfamily\Anaconda3\envs\python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:37:19] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
clf_lr = LR().fit(X_train, y_train)
accuracy_xgb = float(np.sum(preds_xgb == y_test))/y_test.shape[0]
accuracy_lr = clf_lr.score(X_test, y_test)

In [17]:
print('Accuracy de XGBoost: ', accuracy_xgb)


Accuracy de XGBoost:  0.951048951048951


In [14]:
print('Accuracy de Regresion Lineal: ', accuracy_lr)

Accuracy de Regresion Lineal:  0.7222686011972117


Accuracy de XGBoost:  0.958041958041958

Accuracy de Regresion Lineal:  0.7222686011972117